In [54]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.layers import Conv2D, GlobalAveragePooling2D, Input, Dropout, Dense
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.callbacks import Callback, TensorBoard
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import defaultdict
from matplotlib.pyplot import imshow
from PIL import Image
import datetime
import numpy as np
import os, glob, io
import base64
%matplotlib inline

In [55]:
sess = tf.Session()
graph = tf.get_default_graph()
set_session(sess)

In [56]:
batch_size = 32
test_train_split = 0.2
max_epoch = 1
dropout_prob = 0.3
shape = (224, 224)
train_size_per_label = 500
test_size_per_label = 100
test_train_split=0.2
image_path = "/Users/adammenges/Development/notebooks/basicClassifier/houses_120px_classes"

In [57]:
def resize(arr, shape):
    return np.array(Image.fromarray(arr).resize(shape))

def decode_img(msg):
#     msg = msg[msg.find(b"<plain_txt_msg:img>")+len(b"<plain_txt_msg:img>"):
#               msg.find(b"<!plain_txt_msg>")]
    msg = base64.b64decode(msg)
    buf = io.BytesIO(msg)
    img = Image.open(buf)
    return img

def preprocess(arr, shape=(224, 224)):
    arr = np.array([resize(arr[i], shape) for i in range(0, len(arr))]).astype('float32')
    arr = preprocess_input(arr)
    return arr
    
def get_local_images():
    classes = os.listdir(image_path)
    input_arr = []
    target_labels = []
    for class_idx in range(len(classes)):
        paths = glob.glob(os.path.join(image_path, classes[class_idx]) + "/*.png")
        for img_path in tqdm(paths, desc=f'Processing label {classes[class_idx]}: '):
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
#             x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            target_labels.append(class_idx)
            input_arr.append(x)
    X_train, X_test, y_train, y_test = train_test_split(input_arr, target_labels, test_size=test_train_split)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test, classes

def get_cifar10():
    (input_train, out_train), (input_test, out_test) = cifar10.load_data()
    return input_train, input_test, out_train, out_test, range(10)

def get_resnet50(shape=(224, 224, 3)):
    return ResNet50(weights='imagenet', include_top=False, input_shape=shape)

def restrain_data(input_train, out_train, input_test, out_test, num_class, num_train, num_test, shape=(224, 224)):
    train_dict = defaultdict(list)
    test_dict = defaultdict(list)
    [train_dict[out_train[idx][0]].append(input_train[idx]) for idx in range(input_train.shape[0])]
    [test_dict[out_test[idx][0]].append(input_test[idx]) for idx in range(input_test.shape[0])]
    restrain_class = range(num_class)
    restrain_train = [[train_dict[i][idx], i] for idx in range(num_train) for i in restrain_class]
    restrain_test = [[test_dict[i][idx], i] for idx in range(num_test) for i in restrain_class]
    rand_train_idx = np.random.choice(num_train * num_class, num_train * num_class)
    rand_test_idx = np.random.choice(num_test * num_class, num_test * num_class)
    i_train = np.array([restrain_train[idx][0] for idx in rand_train_idx])
    o_train =  np.array([[restrain_train[idx][1]] for idx in rand_train_idx])
    i_test = np.array([restrain_test[idx][0] for idx in rand_test_idx])
    o_test =  np.array([[restrain_test[idx][1]] for idx in rand_test_idx])
    i_train = preprocess(i_train, shape=shape)
    i_test = preprocess(i_test, shape=shape)
    return i_train, i_test, o_train, o_test, restrain_class

In [58]:
input_train, input_test, out_train, out_test, classes = get_local_images()

Processing label Dogtrot: 100%|██████████| 46/46 [00:00<00:00, 1264.12it/s]


In [59]:
input_test.shape

(61, 224, 224, 3)

In [60]:
x = get_cifar10()
x[0].shape

(50000, 32, 32, 3)

In [61]:
# input_train, input_test, out_train, out_test, classes = restrain_data(
#     input_train, 
#     out_train, 
#     input_test,
#     out_test, 
#     len(classes),
#     train_size_per_label,
#     test_size_per_label)
# input_train = preprocess(input_train, shape=shape)
# input_test = preprocess(input_test, shape=shape)

In [62]:
total_train_steps = len(input_train) // batch_size
out_train = to_categorical(out_train, len(classes))
out_test = to_categorical(out_test, len(classes))

In [63]:
def batch_generator(x, y, batch_size=32):
    while True:
        for step in range(len(x) // batch_size):
            yield x[step*batch_size:(step+1)*batch_size, ...], y[step*batch_size:(step+1)*batch_size, ...]

class RecordAccuracy(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f'Running epoch {epoch}. Total {total_train_steps} batches')
    def on_batch_end(self, batch, logs=None):
        loss = logs['loss']
        if not batch % 10:
            print(f'Running batch {batch}: train loss - {loss}')
    def on_epoch_end(self, epoch, logs=None):
        loss = logs["loss"]
        val_acc = logs["val_acc"]
        print(f'Epoch {epoch}: train loss - {loss}. test accuracy - {val_acc}')
        
def freeze_layers(model, layer_num):
    for layer in model.layers[:layer_num]:
        layer.trainable = False
            
def train_layers(model, layer_num):
    for layer in model.layers[layer_num:]:
        layer.trainable = True

In [64]:
resnet50 = get_resnet50(shape=shape + (3,))
bottleneck_train_features = resnet50.predict(input_train)
bottleneck_test_features = resnet50.predict(input_test)

/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [65]:
in_layer = Input(shape=(bottleneck_train_features.shape[1:]))
x = Conv2D(filters=100, kernel_size=2)(in_layer)
x = Dropout(0.4)(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=in_layer, outputs=predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 7, 7, 2048)]      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 100)         819300    
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 100)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 100)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 819,906
Trainable params: 819,906
Non-trainable params: 0
_____________________________________________________

## Train the model!

And now it's time to train the model!

In [66]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit_generator(batch_generator(bottleneck_train_features, out_train),
                    steps_per_epoch=len(bottleneck_train_features) // batch_size,
                      validation_data=(bottleneck_test_features, out_test), 
                      verbose=2,
                      epochs=max_epoch,
                      callbacks=[RecordAccuracy(), TensorBoard()])

Running epoch 0. Total 7 batches
Running batch 0: train loss - 2.5737788677215576
Epoch 0: train loss - 4.872777155467442. test accuracy - 0.7540983557701111
7/7 - 2s - loss: 4.8728 - acc: 0.4152 - val_loss: 1.5288 - val_acc: 0.7541


# Server

Okay now let's host a server for grasshopper

In [67]:
print(model.predict(resnet50.predict(np.array([input_test[0]]))))
print(classes)
print('----')
print(input_test[0].shape)
print(list(zip(model.predict(resnet50.predict(np.array([input_test[0]])))[0], classes)))
out_test[0]

[[1.4172211e-22 9.7809210e-30 1.0000000e+00 5.7962686e-21 2.2097266e-24
  8.6659401e-16]]
['Saltbox', 'QueenAnne', 'AFrame', 'Patio', 'BayGable', 'Dogtrot']
----
(224, 224, 3)
[(1.417221e-22, 'Saltbox'), (9.780921e-30, 'QueenAnne'), (1.0, 'AFrame'), (5.7962686e-21, 'Patio'), (2.2097266e-24, 'BayGable'), (8.66594e-16, 'Dogtrot')]


array([0., 0., 1., 0., 0., 0.], dtype=float32)

In [ ]:
from flask import Flask
from flask import request
app = Flask(__name__)

@app.route('/predict', methods=['POST']) #GET requests will be blocked
def hello_world():
    req_data = request.get_json()
    img = req_data['image']
    img = decode_img(img).resize((224,224)).convert('RGB')
    img = image.img_to_array(img)
    x = preprocess_input(img)
    print('----')
    print(x.shape)
    print('----')
    global sess
    global graph
    with graph.as_default():
        set_session(sess)
        pred = model.predict(resnet50.predict(np.array([x])))[0]
        pred = [str(f) for f in pred]
        prediction = list(zip(pred, classes))
    print('prediction')
    print(prediction)
    return {
        'prediction': prediction
    }

app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Sep/2019 17:42:02] "POST /predict HTTP/1.1" 200 -


----
(224, 224, 3)
----
prediction
[('0.00022993507', 'Saltbox'), ('2.664498e-05', 'QueenAnne'), ('0.99533844', 'AFrame'), ('3.6225214e-05', 'Patio'), ('9.786252e-06', 'BayGable'), ('0.0043591107', 'Dogtrot')]
